# Implementation of model.earth page - Part 1

Using the [write up](https://model.earth/localsite/info/data/) by Catherine from EPA as a guide.

With modifications from Loren: `add display of counties for Motor Vehicle Manufacturing as both 4-digit 3361 and 6-digit 336111. Let’s look at Georgia, Alabama, California and Nevada.`

In [1]:
# if running in colab uncomment next line
#!pip install git+https://github.com/modelearth/flowsa -q

In [2]:
import flowsa
import pandas as pd

## getFlowByActivity

This is the API function we'll be using.  They have docstrings to look at.

In [3]:
help(flowsa.getFlowByActivity)

Help on function getFlowByActivity in module flowsa:

getFlowByActivity(flowclass, years, datasource, geographic_level='all', file_location='local')
    Retrieves stored data in the FlowByActivity format
    :param flowclass: list, a list of`Class' of the flow. required. E.g. ['Water'] or
     ['Land', 'Other']
    :param year: list, a list of years [2015], or [2010,2011,2012]
    :param datasource: str, the code of the datasource.
    :param geographic_level: 'all', 'national', 'state', 'county'. Default is 'all'
    :param file_location: 'local' or 'remote'. Default is 'local'
    :return: a pandas DataFrame in FlowByActivity format



## Use API - returns dataframe

In [4]:
df = flowsa.getFlowByActivity(flowclass=['Employment', 'Money', 'Other'], years=[2018], datasource="BLS_QCEW")

2021-02-24 07:20:39 INFO     BLS_QCEW_2018.parquet not found in local folder; loading from remote server...


## Some standard dataframe analysis.
  1.  Size
  2.  dtypes
  3.  isnull values
  4.  ...
  
I'll go through it my way for the first df.  You can apply your favorite techniques (i.e. plots, ...)

In [5]:
df.shape

(8119338, 20)

In [6]:
#info function in colab doesn't support this parameter
#df.info(show_counts=True) 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8119338 entries, 0 to 8119337
Data columns (total 20 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Class               8119338 non-null  object 
 1   SourceName          8119338 non-null  object 
 2   FlowName            8119338 non-null  object 
 3   FlowAmount          8119338 non-null  float64
 4   Unit                8119338 non-null  object 
 5   FlowType            8119338 non-null  object 
 6   ActivityProducedBy  8119338 non-null  object 
 7   ActivityConsumedBy  0 non-null        object 
 8   Compartment         0 non-null        object 
 9   Location            8119338 non-null  object 
 10  LocationSystem      8119338 non-null  object 
 11  Year                8119338 non-null  int32  
 12  MeasureofSpread     0 non-null        object 
 13  Spread              8119338 non-null  float64
 14  DistributionType    0 non-null        object 
 15  Min            

In [7]:
df.isnull().sum()

Class                       0
SourceName                  0
FlowName                    0
FlowAmount                  0
Unit                        0
FlowType                    0
ActivityProducedBy          0
ActivityConsumedBy    8119338
Compartment           8119338
Location                    0
LocationSystem              0
Year                        0
MeasureofSpread       8119338
Spread                      0
DistributionType      8119338
Min                         0
Max                         0
DataReliability             0
DataCollection              0
Description           8119338
dtype: int64

### FIPS codes

This df was generated with the default geographic_level (all) meaning it returns both state and county FIPS values.  Let's check that out for GA (FIPS = 13000).

In [8]:
df[df.Location.str.startswith('13')]['Location'].unique()

array(['13000', '13001', '13003', '13005', '13007', '13009', '13011',
       '13013', '13015', '13017', '13019', '13021', '13023', '13025',
       '13027', '13029', '13031', '13033', '13035', '13037', '13039',
       '13043', '13045', '13047', '13049', '13051', '13053', '13055',
       '13057', '13059', '13061', '13063', '13065', '13067', '13069',
       '13071', '13073', '13075', '13077', '13079', '13081', '13083',
       '13085', '13087', '13089', '13091', '13093', '13095', '13097',
       '13099', '13101', '13103', '13105', '13107', '13109', '13111',
       '13113', '13115', '13117', '13119', '13121', '13123', '13125',
       '13127', '13129', '13131', '13133', '13135', '13137', '13139',
       '13141', '13143', '13145', '13147', '13149', '13151', '13153',
       '13155', '13157', '13159', '13161', '13163', '13165', '13167',
       '13169', '13171', '13173', '13175', '13177', '13179', '13181',
       '13183', '13185', '13187', '13189', '13191', '13193', '13195',
       '13197', '131

In [9]:
len(_)

161

There are 159 counties in GA according to wikipedia.  Since the count is including the state fips and there looks to be a weird one, 13999, I think it's good.

In [10]:
#df = flowsa.getFlowByActivity(flowclass=['Employment', 'Money', 'Other'], years=[2015, 2016], datasource="BLS_QCEW")

In [11]:
from flowsa.common import fbaoutputpath

In [12]:
fc=['Employment', 'Money', 'Other']
years=[2018]
ds="BLS_QCEW"

## County level data
Digressing a bit from the write up on model.earth at this point.

In [13]:
# using the county geographic_level
county_df = flowsa.getFlowByActivity(flowclass=fc, years=years, datasource=ds, geographic_level='county')

2021-02-24 07:20:55 INFO     BLS_QCEW_2018.parquet not found in local folder; loading from remote server...


In [14]:
county_df.shape

(7584573, 20)

## Build dataframe with state specific rows.

In [15]:
# Create new column with first two digits of Location representing state
county_df = county_df.assign(StateFIPS=county_df['Location'].apply(lambda x: x[0:2]))

In [16]:
# create subset with the states, AL, GA, CA, NV
state_subset_df = county_df[county_df['StateFIPS'].isin(['01', '06', '13', '32'])].reset_index(drop=True)

In [17]:
state_subset_df.shape

(742038, 21)

In [18]:
state_subset_df['StateFIPS'].value_counts()

13    322428
06    225264
01    157185
32     37161
Name: StateFIPS, dtype: int64

So this makes sense.

## Further filter for NAICS of interest

Based on discussion with Loren I am simplifying to auto manufacturing type codes.

In [19]:
len(state_subset_df.ActivityProducedBy.unique())

2229

In [20]:
state_subset_df_auto = state_subset_df[state_subset_df['ActivityProducedBy'].isin(['3361', '336111'])].reset_index(drop=True)

In [21]:
len(state_subset_df_auto)

237

We now have a very managable data set to understand how to roll things around.  It contains information on two NAICS codes and 4 states.

## Transform df to generate required csv

I am going to simplify and just look at GA for starters and a subset of the columns.

In [22]:
georgia_df = state_subset_df_auto.query(f"StateFIPS == '13'").reset_index(drop=True)[['Location', 'ActivityProducedBy', 'Class', 'FlowAmount']]

In [23]:
georgia_df = georgia_df.rename(columns={'Location': 'fips', 'ActivityProducedBy': 'naics'})

Final prep on the df before we group and generate the df that can be used for the csv file.

In [24]:
new_column_names = {'Employment': 'employees', 'Money': 'wages', 'Other': 'firms'}

georgia_df['Class'] = georgia_df['Class'].apply(lambda r: new_column_names[r])

Let's take a peek at the dataframe now.

Finally, this code snippet groups the data, iterates, and builds the final df.  Someone can show me if there's a `simple` pandas method to do this?

In [25]:
rows = []
cols = ['fips', 'naics', 'employees', 'firms', 'wages']
grouped_df = georgia_df.groupby(['fips', 'naics', 'Class'])['FlowAmount'].sum().unstack()
for index, row in grouped_df.iterrows():
    rows.append([index[0], index[1], row.employees, row.firms, row.wages])
    
pd.DataFrame(data=rows, columns=cols)

,fips,naics,employees,firms,wages
0,13013,3361,0.0,1.0,0.0
1,13013,336111,0.0,1.0,0.0
2,13067,3361,0.0,1.0,0.0
3,13089,3361,0.0,1.0,0.0
4,13089,336111,0.0,1.0,0.0
5,13119,3361,0.0,1.0,0.0
6,13119,336111,0.0,1.0,0.0
7,13121,3361,0.0,3.0,0.0
8,13121,336111,0.0,3.0,0.0
9,13135,3361,0.0,1.0,0.0


Whole lot of 0.0??

In [26]:
georgia_df.query(f"Class == 'wages'")

,fips,naics,Class,FlowAmount
24,13013,3361,wages,0.0
25,13013,336111,wages,0.0
26,13067,3361,wages,0.0
27,13089,3361,wages,0.0
28,13089,336111,wages,0.0
29,13119,3361,wages,0.0
30,13119,336111,wages,0.0
31,13121,3361,wages,0.0
32,13121,336111,wages,0.0
33,13135,3361,wages,0.0


##  This is the end of Part 1.

In the next edition I am going to explore state info vs county info and generate the output files.